In [2]:
#Import neccesary packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [1]:
#Install LASER embedding package
!pip install laserembeddings

     |████████████████████████████████| 51kB 7.3MB/s 
     |████████████████████████████████| 860kB 34.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp37-none-any.whl size=883999 sha256=563504d364a107fa9be9460a32e3136e56dae6282ad8cef9dfe0322f104e790e
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [4]:
raw_data = pd.read_csv('/gdrive/MyDrive/CMPE257_MachineLearning/Lab3/train.csv')
raw_data.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1


In [5]:
raw_test = pd.read_csv('/gdrive/MyDrive/CMPE257_MachineLearning/Lab3/test.csv')
raw_combined = pd.concat([raw_data, raw_test])
raw_combined.head()

,id,premise,hypothesis,lang_abv,language,label
0,5130fd2cb5,and these comments were considered in formulat...,The rules developed in the interim were put to...,en,English,0.0
1,5b72532a0b,These are issues that we wrestle with in pract...,Practice groups are not permitted to work on t...,en,English,2.0
2,3931fbe82a,Des petites choses comme celles-là font une di...,J'essayais d'accomplir quelque chose.,fr,French,0.0
3,5622f0c60b,you know they can't really defend themselves l...,They can't defend themselves because of their ...,en,English,0.0
4,86aaa48b45,ในการเล่นบทบาทสมมุติก็เช่นกัน โอกาสที่จะได้แสด...,เด็กสามารถเห็นได้ว่าชาติพันธุ์แตกต่างกันอย่างไร,th,Thai,1.0


In [ ]:
#Install other languages that do not come with the standard LASER package
import os
for l in raw_combined.lang_abv.unique():
    os.system(f'!pip install laserembeddings[{l}]')

In [ ]:
#Download the pre-train model for LASER embedding
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
#Init LASER & Embed the sentences
from laserembeddings import Laser
laser = Laser()

vectors = []

for i, r in raw_combined.iterrows():
    vectors.append(
        np.concatenate(
            (
                laser.embed_sentences(r.hypothesis, lang=r.lang_abv)[0], 
                laser.embed_sentences(r.premise, lang=r.lang_abv)[0],
            )
        )
    )
vectors = np.array(vectors)

In [6]:
#Load the embedded vectors that is pickled so that we don't have to waste time re-embed the whole dataset
import pickle
pickle_in=open("/gdrive/MyDrive/CMPE257_MachineLearning/Lab3/vector.pickle","rb")
vectors = pickle.load(pickle_in)
pickle_in.close()

In [ ]:
#This is how the embedded vectors look like
vectors

array([[ 7.2747874e-03,  1.3423607e-03, -5.0730145e-05, ...,
         1.2217250e-02,  9.3295351e-03,  3.4985673e-02],
       [ 1.0637566e-02, -1.2596670e-04,  9.6913980e-04, ...,
         1.2963601e-02,  1.1006403e-02,  2.1777369e-02],
       [ 7.9386216e-03,  1.5886479e-05,  1.4029618e-04, ...,
         6.6946782e-03,  2.1546386e-02,  2.7466459e-02],
       ...,
       [ 3.0137517e-03, -4.4617351e-04,  7.3791202e-04, ...,
         1.8130897e-02,  1.8045582e-02,  1.7430648e-02],
       [ 7.7674584e-03, -4.2668344e-06, -1.4443031e-04, ...,
         8.3482042e-03,  5.7175830e-03,  1.3380240e-02],
       [ 4.9003786e-03, -4.5182951e-05,  8.9579378e-04, ...,
         1.9976727e-03,  1.0322244e-02,  3.3516090e-02]], dtype=float32)

In [7]:
#Prepare data for training and testing since we combined them together earlier to embed

X = vectors[:len(raw_data)]
y = raw_data.label.to_numpy()
test = vectors[len(raw_data):]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [8]:
#We tranform the y using OneHotEncoder to feed into the SoftMax layer
from keras.utils import to_categorical
y_train = to_categorical(y_train, 3)
y_test = to_categorical(y_test, 3)

In [ ]:
X_train.shape

(8484, 2048)

In [ ]:
y_train.shape

(8484,)

In [ ]:
#First attempt: Using a basic Naive Bayes model
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
# predict class
y_pred = classifier.predict(X_test)

In [ ]:
# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(cr)
print(accuracy)

[[344 520 414]
 [214 639 318]
 [184 395 608]]
              precision    recall  f1-score   support

           0       0.46      0.27      0.34      1278
           1       0.41      0.55      0.47      1171
           2       0.45      0.51      0.48      1187

    accuracy                           0.44      3636
   macro avg       0.44      0.44      0.43      3636
weighted avg       0.44      0.44      0.43      3636

0.4375687568756876


In [ ]:
#Attempt 2: Init NN model with many deep layers

from keras import layers
from keras import Sequential
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization
input_dim = X_train.shape[1]

model=Sequential()
model.add(Dense(2048, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(2048, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(512, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(256, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(128, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(64, input_dim = input_dim, activation="relu"))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Dense(3, activation='softmax', input_dim = input_dim))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
dropout_30 (Dropout)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 2048)              8192      
_________________________________________________________________
dense_36 (Dense)             (None, 2048)              4196352   
_________________________________________________________________
dropout_31 (Dropout)         (None, 2048)              0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 2048)              8192      
_________________________________________________________________
dense_37 (Dense)             (None, 512)              

In [ ]:
# Evaluate the model
#The accuracy is not that much different from the basic Naieve Bayes model
loss, accuracy = model.evaluate(X_test, y_test)

114/114 [==============================] - 0s 3ms/step - loss: 4.7152 - accuracy: 0.4607


In [16]:
#Attempt 3: simplify the model as we trust the LASER embedding alrady mapped the sentences into the same neighborhood, thus we don;t have to overcomplicated our classifier

from keras import layers
from keras import Sequential
from keras.layers import Dense
input_dim = X_train.shape[1]

model=Sequential()
model.add(Dense(3, activation='softmax', input_dim = input_dim))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 3)                 6147      
Total params: 6,147
Trainable params: 6,147
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
#Train the model
model.fit(X_train, y_train, batch_size = 512 ,epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
17/17 [==============================] - 1s 16ms/step - loss: 1.0982 - accuracy: 0.3436 - val_loss: 1.0952 - val_accuracy: 0.3740
Epoch 2/100
17/17 [==============================] - 0s 8ms/step - loss: 1.0927 - accuracy: 0.3844 - val_loss: 1.0914 - val_accuracy: 0.4139
Epoch 3/100
17/17 [==============================] - 0s 7ms/step - loss: 1.0882 - accuracy: 0.4391 - val_loss: 1.0883 - val_accuracy: 0.4266
Epoch 4/100
17/17 [==============================] - 0s 8ms/step - loss: 1.0844 - accuracy: 0.4578 - val_loss: 1.0856 - val_accuracy: 0.4356
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 1.0793 - accuracy: 0.4930 - val_loss: 1.0829 - val_accuracy: 0.4345
Epoch 6/100
17/17 [==============================] - 0s 7ms/step - loss: 1.0754 - accuracy: 0.4815 - val_loss: 1.0809 - val_accuracy: 0.4450
Epoch 7/100
17/17 [==============================] - 0s 7ms/step - loss: 1.0727 - accuracy: 0.4880 - val_loss: 1.0783 - val_accuracy: 0.4453
Epoch 8/100


In [22]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)

114/114 [==============================] - 0s 2ms/step - loss: 0.9954 - accuracy: 0.5014


In [19]:
#Save the model to use in Kaggle
from keras.models import load_model

model.save('Laser_NN_model_new.h5')

In [23]:
#Predict the test set
model.predict(test)

array([[0.38727045, 0.30288032, 0.30984917],
       [0.21300131, 0.5258806 , 0.2611181 ],
       [0.33013898, 0.42802125, 0.24183981],
       ...,
       [0.2568004 , 0.4456846 , 0.29751503],
       [0.399247  , 0.26443934, 0.33631366],
       [0.35334837, 0.27687955, 0.36977208]], dtype=float32)

In [24]:
result = model.predict(test)

In [ ]:
#Convert the data back into category
result_in_cat = np.argmax(result, axis=-1)

In [ ]:
result_in_cat

array([0, 1, 1, ..., 1, 2, 0])

In [ ]:
result_in_cat.shape

(5195,)

In [ ]:
#Get the ID from test set
raw_test.loc[:,"id"]

0       c6d58c3f69
1       cefcc82292
2       e98005252c
3       58518c10ba
4       c32b0d16df
           ...    
5190    5f90dd59b0
5191    f357a04e86
5192    1f0ea92118
5193    0407b48afb
5194    16c2f2ab89
Name: id, Length: 5195, dtype: object

In [ ]:
result_np = np.array(result_in_cat)

In [ ]:
#Convert the prediction into a Pandas df for easier operation
result_df = pd.DataFrame(result_np, columns = ['prediction'])

In [ ]:
result_df

,prediction
0,0
1,1
2,1
3,0
4,2
...,...
5190,1
5191,1
5192,1
5193,2


In [ ]:
#Create final submission file for LASER
file_result = pd.concat([raw_test.loc[:,"id"],result_df], axis=1)

In [ ]:
file_result

,id,prediction
0,c6d58c3f69,0
1,cefcc82292,1
2,e98005252c,1
3,58518c10ba,0
4,c32b0d16df,2
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,2


In [ ]:
file_result.to_csv('submission.csv')
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>